In [3]:
import DBcm

In [4]:
config = {
    "user": "swimuser",
    "password": "swimuserpasswd",
    "database": "SwimclubDB",
    "host": "localhost",
}

In [6]:
with DBcm.UseDatabase(config) as db:
    SQL = """ show tables """
    db.execute(SQL)
    results = db.fetchall()

print(results)

[('events',), ('swimmers',), ('times',)]


In [ ]:
def upload_data(file_name: str) -> None:
    with DBcm.UseDatabase(config) as db:
        SQL = """   INSERT INTO events
                    (name, age, location)
                    VALUES
                    ("Paul", 21, "Carlow")
            """